In [ ]:

from myocr.pipelines.common_ocr_pipeline import CommonOCRPipeline

common_ocr_pipeline = CommonOCRPipeline("cuda:0")
res = common_ocr_pipeline("../../tests/images/invoice.png")
print(res.to_dict())

2025-04-24 16:23:23,941 - myocr.modeling.model - INFO - model - load - 127 - Onnx model /home/robby/.MyOCR/models/dbnet++.onnx loaded to cuda:0,
                    input output info: {'inputs': {'x': {'shape': ['p2o.DynamicDimension.0', 3, 'p2o.DynamicDimension.1', 'p2o.DynamicDimension.2'], 'type': 'tensor(float)'}}, 'outputs': {'sigmoid_0.tmp_0': {'shape': ['p2o.DynamicDimension.3', 1, 'p2o.DynamicDimension.4', 'p2o.DynamicDimension.5'], 'type': 'tensor(float)'}}}
2025-04-24 16:23:23,953 - myocr.modeling.model - INFO - model - load - 127 - Onnx model /home/robby/.MyOCR/models/cls.onnx loaded to cuda:0,
                    input output info: {'inputs': {'x': {'shape': ['p2o.DynamicDimension.0', 3, 'p2o.DynamicDimension.1', 'p2o.DynamicDimension.2'], 'type': 'tensor(float)'}}, 'outputs': {'softmax_0.tmp_0': {'shape': ['p2o.DynamicDimension.3', 2], 'type': 'tensor(float)'}}}
2025-04-24 16:23:23,992 - myocr.modeling.model - INFO - model - load - 127 - Onnx model /home/robby/.MyOCR/model